In [88]:
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', None)

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

In [89]:
submission_csv = pd.read_csv('test_data/Professor_CSV.csv', keep_default_na=False)
former_flights_data = pd.read_csv('dataset/merged_data/former_flight_data.csv')
latter_flight_data = pd.read_csv('dataset/merged_data/latter_flight_data.csv')
test_data = pd.read_csv('test_data/Test_Merged_Data.csv')


In [90]:
# Drop NaN for latter flight data
if True:
    latter_flight_data.dropna(subset=['FORMER_FLIGHT_STATUS'], inplace=True)

In [91]:
def categorize_delay(delay):
    if delay < -7:
        return 0 #early
    elif delay > 7:
        return 2 #late
    else:
        return 1 #ontime

In [92]:
X_former_flights_data = former_flights_data.drop(columns=['ARR_DELAY'])
y_former_flights_data = former_flights_data['ARR_DELAY'].apply(categorize_delay)

X_latter_flight_data = latter_flight_data.drop(columns=['ARR_DELAY'])
y_latter_flight_data = latter_flight_data['ARR_DELAY'].apply(categorize_delay)

In [93]:
# Handle Categorical Variables
categorical_vars = ['DAY_OF_WEEK', 'MKT_UNIQUE_CARRIER',
                    'OP_UNIQUE_CARRIER', 'ORIGIN',
                    'ORGIN_WTH_precipprob', 'ORGIN_WTH_severerisk',
                    'DEST_WTH_precipprob', 'DEST_WTH_severerisk',
                    'FORMER_FLIGHT_STATUS', 'MONTH']

# categorical_vars = potential_categorical_vars.keys()

In [94]:
def preprocess(flight_data: pd.DataFrame):

    # Dealing with date and time
    flight_data['SCH_ARR_TIME'] = pd.to_datetime(flight_data['SCH_ARR_TIME'])
    flight_data['SCH_DEP_TIME'] = pd.to_datetime(flight_data['SCH_DEP_TIME'])

    flight_data['MONTH'] = flight_data['SCH_ARR_TIME'].dt.month
    flight_data['DAY'] = flight_data['SCH_ARR_TIME'].dt.day
    flight_data['DEP_MINUTES'] = flight_data['SCH_DEP_TIME'].dt.hour * 60 + flight_data['SCH_DEP_TIME'].dt.minute
    flight_data['ARR_MINUTES'] = flight_data['SCH_ARR_TIME'].dt.hour * 60 + flight_data['SCH_ARR_TIME'].dt.minute

    flight_data.drop(columns=['SCH_DEP_TIME', 'SCH_ARR_TIME'], inplace=True)

    # Dropping unwanted columns
    cols = [
        # 'ORGIN_WTH_temp', 'DEST_WTH_temp',
        'DEST_WTH_severerisk', 'ORGIN_WTH_severerisk',
        'DEST_WTH_precipprob', 'ORGIN_WTH_precipprob'
        ]
    flight_data.drop(columns=cols, inplace=True)
    
    cat_col = list(set(flight_data.columns).intersection(categorical_vars))
    flight_data = pd.get_dummies(flight_data, columns = list(cat_col), drop_first = False)

    return flight_data
    

In [95]:
latter_flight_data.head()


,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,ARR_DELAY,SCH_DEP_TIME,SCH_ARR_TIME,ORGIN_WTH_temp,ORGIN_WTH_precip,ORGIN_WTH_precipprob,ORGIN_WTH_snow,ORGIN_WTH_windspeed,ORGIN_WTH_winddir,ORGIN_WTH_cloudcover,ORGIN_WTH_visibility,ORGIN_WTH_severerisk,DEST_WTH_temp,DEST_WTH_precip,DEST_WTH_precipprob,DEST_WTH_snow,DEST_WTH_windspeed,DEST_WTH_winddir,DEST_WTH_cloudcover,DEST_WTH_visibility,DEST_WTH_severerisk,FORMER_FLIGHT_STATUS
2,6,B6,B6,MCO,22.00,2022-01-01 13:13:00,2022-01-01 15:56:00,83.00,0.00,0,0.00,9.90,199.00,4.30,9.90,3.00,47.70,0.00,0,0.00,7.90,311.00,100.00,7.80,3.00,early
6,7,UA,OO,ORD,48.00,2022-01-02 10:40:00,2022-01-02 13:32:00,23.50,0.00,0,0.00,11.30,330.00,90.30,6.70,3.00,23.00,0.00,0,0.01,11.20,301.00,100.00,8.50,3.00,late
7,7,DL,9E,JFK,180.00,2022-01-02 12:55:00,2022-01-02 14:12:00,57.20,0.00,0,0.00,10.10,243.00,90.70,9.40,3.00,23.00,0.00,0,0.01,11.20,301.00,100.00,8.50,3.00,early
9,7,AA,MQ,ORD,35.00,2022-01-02 17:25:00,2022-01-02 20:12:00,25.70,0.00,0,0.00,6.40,329.00,24.20,9.90,3.00,24.00,0.00,0,0.01,6.80,338.00,99.90,9.90,3.00,late
10,7,UA,OO,ORD,136.00,2022-01-02 17:55:00,2022-01-02 20:52:00,24.50,0.00,0,0.00,7.70,315.00,24.20,9.90,3.00,21.70,0.00,0,0.01,6.90,343.00,98.60,8.80,3.00,late


In [96]:
# X = preprocess(X)
X_former_flights_data = preprocess(X_former_flights_data)
X_latter_flight_data = preprocess(X_latter_flight_data)
test_data = preprocess(test_data)

In [97]:
# Get missing columns in the prediction data
missing_cols = set(X_former_flights_data.columns) - set(test_data.columns)
# Add a zero column for missing columns in prediction data
for c in missing_cols:
    test_data[c] = 0

# Ensure the order of columns in prediction data matches that of flight_data_encoded
test_data = test_data[X_former_flights_data.columns]
test_data.columns
test_data.shape
# Now, prediction_data_encoded should have the same columns as flight_data_encoded

Index(['ORGIN_WTH_temp', 'ORGIN_WTH_precip', 'ORGIN_WTH_snow',
       'ORGIN_WTH_windspeed', 'ORGIN_WTH_winddir', 'ORGIN_WTH_cloudcover',
       'ORGIN_WTH_visibility', 'DEST_WTH_temp', 'DEST_WTH_precip',
       'DEST_WTH_snow', 'DEST_WTH_windspeed', 'DEST_WTH_winddir',
       'DEST_WTH_cloudcover', 'DEST_WTH_visibility', 'DAY', 'DEP_MINUTES',
       'ARR_MINUTES', 'MONTH_1', 'MONTH_2', 'MONTH_3', 'MONTH_4', 'MONTH_5',
       'MONTH_6', 'MONTH_7', 'MONTH_8', 'MONTH_9', 'MONTH_10', 'MONTH_11',
       'MONTH_12', 'DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3',
       'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5', 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7',
       'OP_UNIQUE_CARRIER_9E', 'OP_UNIQUE_CARRIER_B6', 'OP_UNIQUE_CARRIER_G7',
       'OP_UNIQUE_CARRIER_MQ', 'OP_UNIQUE_CARRIER_OO', 'OP_UNIQUE_CARRIER_PT',
       'OP_UNIQUE_CARRIER_UA', 'OP_UNIQUE_CARRIER_WN', 'OP_UNIQUE_CARRIER_YX',
       'OP_UNIQUE_CARRIER_ZW', 'MKT_UNIQUE_CARRIER_AA',
       'MKT_UNIQUE_CARRIER_B6', 'MKT_UNIQUE_CARRIER_DL',
       

(51, 54)

In [98]:
# Check if PCA using SVD gives same results
from sklearn.decomposition import PCA
# initializing the PCA transformer
pca_former = PCA(n_components = 15)
# dimensionality reduction:
data_pca_former = pd.DataFrame(pca_former.fit_transform(X_former_flights_data), index = X_former_flights_data.index)
data_pca_former.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,348.85,-94.94,-41.00,34.76,0.92,-32.37,63.38,-14.47,-7.89,-5.89,0.85,-1.24,0.28,0.04,1.13
1,-241.21,-119.50,-87.78,23.79,-20.42,18.39,71.02,-13.48,-1.58,-3.58,1.02,-0.48,0.27,-0.07,1.13
2,-77.48,-92.86,-44.09,20.86,-13.79,4.67,63.95,-13.50,-0.56,-2.22,3.77,-0.69,0.23,-0.11,1.12
3,-19.55,-90.46,-42.94,20.14,-15.90,3.67,64.96,-13.51,-0.34,-2.39,3.74,-0.70,0.23,-0.06,-0.52
4,-555.05,-5.18,247.40,45.25,50.60,-12.85,32.67,-14.32,2.65,-3.56,4.88,-0.55,-0.31,0.95,0.06


In [99]:
# Check if PCA using SVD gives same results
from sklearn.decomposition import PCA
# initializing the PCA transformer
pca_latter = PCA(n_components = 15)
# dimensionality reduction:
data_pca_latter = pd.DataFrame(pca_latter.fit_transform(X_latter_flight_data), index = X_latter_flight_data.index)
data_pca_latter.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
2,262.39,-78.94,-51.96,38.78,9.67,-45.44,-2.95,13.52,21.72,1.48,1.22,1.17,0.39,-0.39,1.03
6,473.71,-140.49,60.95,-46.82,-15.02,-2.92,36.58,14.58,0.51,-2.70,-0.28,0.40,2.48,-0.36,-0.42
7,348.08,-92.63,-8.42,-16.66,-55.27,26.66,-4.39,12.33,23.34,3.55,0.81,-0.41,-0.12,0.02,-0.27
9,-90.99,-211.63,14.45,4.85,-5.46,-33.01,39.78,14.59,-2.80,-6.61,-0.89,-1.37,0.52,1.27,-0.07
10,-140.10,-209.79,-1.49,1.94,-1.13,-36.12,44.63,14.55,-2.44,-5.70,-0.24,-0.31,0.28,-0.25,-0.60


In [100]:
# Get missing columns in the prediction data
missing_cols = set(X_former_flights_data.columns) - set(test_data.columns)
# Add a zero column for missing columns in prediction data
for c in missing_cols:
    test_data[c] = 0

# Ensure the order of columns in prediction data matches that of flight_data_encoded
test_data = test_data[X_former_flights_data.columns]
test_data.columns
test_data.shape
# Now, prediction_data_encoded should have the same columns as flight_data_encoded

Index(['ORGIN_WTH_temp', 'ORGIN_WTH_precip', 'ORGIN_WTH_snow',
       'ORGIN_WTH_windspeed', 'ORGIN_WTH_winddir', 'ORGIN_WTH_cloudcover',
       'ORGIN_WTH_visibility', 'DEST_WTH_temp', 'DEST_WTH_precip',
       'DEST_WTH_snow', 'DEST_WTH_windspeed', 'DEST_WTH_winddir',
       'DEST_WTH_cloudcover', 'DEST_WTH_visibility', 'DAY', 'DEP_MINUTES',
       'ARR_MINUTES', 'MONTH_1', 'MONTH_2', 'MONTH_3', 'MONTH_4', 'MONTH_5',
       'MONTH_6', 'MONTH_7', 'MONTH_8', 'MONTH_9', 'MONTH_10', 'MONTH_11',
       'MONTH_12', 'DAY_OF_WEEK_1', 'DAY_OF_WEEK_2', 'DAY_OF_WEEK_3',
       'DAY_OF_WEEK_4', 'DAY_OF_WEEK_5', 'DAY_OF_WEEK_6', 'DAY_OF_WEEK_7',
       'OP_UNIQUE_CARRIER_9E', 'OP_UNIQUE_CARRIER_B6', 'OP_UNIQUE_CARRIER_G7',
       'OP_UNIQUE_CARRIER_MQ', 'OP_UNIQUE_CARRIER_OO', 'OP_UNIQUE_CARRIER_PT',
       'OP_UNIQUE_CARRIER_UA', 'OP_UNIQUE_CARRIER_WN', 'OP_UNIQUE_CARRIER_YX',
       'OP_UNIQUE_CARRIER_ZW', 'MKT_UNIQUE_CARRIER_AA',
       'MKT_UNIQUE_CARRIER_B6', 'MKT_UNIQUE_CARRIER_DL',
       

(51, 54)

In [101]:
former_flight_model = xgb.XGBClassifier(eta = '0.03', max_depth=4, min_child_weight=2, reg_lambda=0.009, n_estimators=600)

former_flight_model = former_flight_model.fit(data_pca_former, y_former_flights_data)
former_flight_model.score(data_pca_former, y_former_flights_data)
feat_imp_former = pd.Series(former_flight_model.feature_importances_, data_pca_former.columns.values).sort_values(ascending=False)
feat_imp_former.head(15)

latter_flight_model = xgb.XGBClassifier(eta = '0.03', max_depth=4, min_child_weight=2, reg_lambda=0.009, n_estimators=600)

latter_flight_model = latter_flight_model.fit(data_pca_latter, y_latter_flight_data)
latter_flight_model.score(data_pca_latter, y_latter_flight_data)
feat_imp_latter = pd.Series(latter_flight_model.feature_importances_, data_pca_latter.columns.values).sort_values(ascending=False)
feat_imp_latter.head(15)


0.7066292632511443

13   0.13
3    0.08
0    0.07
11   0.07
6    0.07
14   0.07
12   0.07
2    0.06
4    0.06
8    0.06
7    0.06
5    0.05
9    0.05
10   0.05
1    0.05
dtype: float32

0.7766272189349113

14   0.08
6    0.08
0    0.08
3    0.07
13   0.07
11   0.07
2    0.07
4    0.07
12   0.07
8    0.06
9    0.06
5    0.06
10   0.06
1    0.06
7    0.06
dtype: float32

In [102]:
status_dic = {0: 'early', 1: 'ontime', 2: 'late'}
for index, sub_row in submission_csv.iterrows():
    test_row = test_data.iloc[index].copy()


    # Predict Former    
    test_df = pd.DataFrame(test_row).transpose()
    former_row = pd.DataFrame(pca_former.transform(test_df), index = test_df.index)
    former = status_dic[former_flight_model.predict(former_row)[0]]
    if sub_row['ARRIVAL STATUS'] != 'NA':
        sub_row['ARRIVAL STATUS'] = former

    # Predict Latter - Former Early
    test_row['FORMER_FLIGHT_STATUS_late'] = 0
    test_row['FORMER_FLIGHT_STATUS_on-time'] = 0
    test_row['FORMER_FLIGHT_STATUS_early'] = 1
    test_df = pd.DataFrame(test_row).transpose()
    test_df = test_df[X_latter_flight_data.columns]
    early_row = pd.DataFrame(pca_latter.transform(test_df), index = test_df.index)
    early = status_dic[latter_flight_model.predict(early_row)[0]]
    if sub_row['ARRIVAL STATUS_Prev_flight_early'] != 'NA':
        sub_row['ARRIVAL STATUS_Prev_flight_early'] = early

    # Predict Latter - Former ontime
    test_row['FORMER_FLIGHT_STATUS_late'] = 0
    test_row['FORMER_FLIGHT_STATUS_on-time'] = 1
    test_row['FORMER_FLIGHT_STATUS_early'] = 0
    test_df = pd.DataFrame(test_row).transpose()
    test_df = test_df[X_latter_flight_data.columns]
    ontime_row = pd.DataFrame(pca_latter.transform(test_df), index = test_df.index)
    ontime = status_dic[latter_flight_model.predict(ontime_row)[0]]
    if sub_row['ARRIVAL STATUS_Prev_flight_ontime'] != 'NA':
        sub_row['ARRIVAL STATUS_Prev_flight_ontime'] = ontime
    
    # Predict Latter - Former late
    test_row['FORMER_FLIGHT_STATUS_late'] = 1
    test_row['FORMER_FLIGHT_STATUS_on-time'] = 0
    test_row['FORMER_FLIGHT_STATUS_early'] = 0
    test_df = pd.DataFrame(test_row).transpose()
    test_df = test_df[X_latter_flight_data.columns]
    late_row = pd.DataFrame(pca_latter.transform(test_df), index = test_df.index)
    late = status_dic[latter_flight_model.predict(late_row)[0]]
    if sub_row['ARRIVAL STATUS_Prev_flight_late'] != 'NA':
        sub_row['ARRIVAL STATUS_Prev_flight_late'] = late

    

In [103]:
import datetime

def get_status(scheduled, actual):
    time_format = '%I:%M %p'  # Format for hours:minutes AM/PM
    scheduled_time = datetime.datetime.strptime(scheduled, time_format)
    actual_time = datetime.datetime.strptime(actual, time_format)
    
    delay = actual_time - scheduled_time
    
    if delay < datetime.timedelta(minutes=-5):
        return 'early'
    elif delay > datetime.timedelta(minutes=5):
        return 'late'
    else:
        return 'ontime'


In [104]:
actual_df = pd.read_csv('./test_data/Actual_Test_Data.csv')
actual_df = actual_df.head(submission_csv.shape[0])
check_correctness_df = submission_csv.merge(actual_df['ACTUAL ARRIVAL TIME'], left_index=True, right_index=True)
check_correctness_df['Actual Status'] = check_correctness_df['ARRIVAL TIME']
check_correctness_df['Actual Status'] = check_correctness_df.apply(lambda row: get_status(row['ARRIVAL TIME'], row['ACTUAL ARRIVAL TIME']), axis=1)

count = 0
for i in range(0, check_correctness_df.shape[0]):
    cur_row = check_correctness_df.iloc[i]

    # former
    if cur_row['ARRIVAL STATUS'] != 'NA':
        if cur_row['ARRIVAL STATUS'] != cur_row['Actual Status']: count += 1
        former_status = cur_row['Actual Status']
    
    # latter
    else:
        col = 'ARRIVAL STATUS_Prev_flight_' + former_status
        if cur_row[col] != cur_row['Actual Status']: count += 1

print(f'Correct: {check_correctness_df.shape[0] - count}\nIncorrect: {count}')

check_correctness_df

Correct: 23
Incorrect: 28


,DATE,DAY,FLIGHT NUMBER,MKT_UNIQUE_CARRIER,OP_UNIQUE_CARRIER,ORIGIN,DEPARTURE TIME,ARRIVAL TIME,ARRIVAL STATUS,ARRIVAL STATUS_Prev_flight_early,ARRIVAL STATUS_Prev_flight_ontime,ARRIVAL STATUS_Prev_flight_late,ACTUAL ARRIVAL TIME,Actual Status
0,4/10/24,WEDNESDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,ontime,NA,NA,NA,9:35 PM,early
1,4/10/24,WEDNESDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NA,early,late,ontime,10:33 PM,early
2,4/10/24,WEDNESDAY,B6 116,B6,B6,JFK,1:33 PM,2:50 PM,late,NA,NA,NA,2:24 PM,early
3,4/10/24,WEDNESDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NA,ontime,ontime,early,3:57 PM,early
4,4/10/24,WEDNESDAY,WN 5285,WN,WN,MCO,11:05 AM,1:45 PM,late,NA,NA,NA,1:29 PM,early
5,4/10/24,WEDNESDAY,B6 656,B6,B6,MCO,1:35 PM,4:25 PM,NA,early,early,early,4:26 PM,ontime
6,4/11/24,THURSDAY,UA 1400,UA,UA,ORD,6:52 PM,9:47 PM,early,NA,NA,NA,9:57 PM,late
7,4/11/24,THURSDAY,AA 3402,AA,MQ,ORD,7:59 PM,10:52 PM,NA,early,ontime,ontime,10:50 PM,ontime
8,4/11/24,THURSDAY,B6 116,B6,B6,JFK,1:33 PM,2:50 PM,late,NA,NA,NA,2:59 PM,late
9,4/11/24,THURSDAY,DL 5182,DL,9E,JFK,2:55 PM,4:21 PM,NA,early,early,early,4:51 PM,late
